# Imports

In [92]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import math
from collections import defaultdict
from tqdm import tqdm
import json
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from collections import defaultdict
import random
import pandas as pd


# Load Data

In [216]:
df = pd.read_csv('../dataset/weighted_score_above_08.csv')

df.head()

/var/folders/sr/kk6cvpy92zvfzcj5gr5l6c6h0000gn/T/ipykernel_12916/1413335316.py:1: DtypeWarning: Columns (23) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../dataset/weighted_score_above_08.csv')


,recommendationid,appid,game,author_steamid,author_num_games_owned,author_num_reviews,author_playtime_forever,author_playtime_last_two_weeks,author_playtime_at_review,author_last_played,...,voted_up,votes_up,votes_funny,weighted_vote_score,comment_count,steam_purchase,received_for_free,written_during_early_access,hidden_in_steam_china,steam_china_location
0,147449116,10,Counter-Strike,76561199183984450,51,12,2548,0,2480,1696305457,...,1,99,2,0.889438,0,1,0,0,1,NaN
1,147374264,10,Counter-Strike,76561198099573060,226,13,2369,0,2361,1696096555,...,1,122,8,0.914834,0,1,0,0,1,NaN
2,147357703,10,Counter-Strike,76561199080026894,118,23,13501,212,12957,1697630734,...,1,599,20,0.968375,3,1,0,0,1,NaN
3,147345102,10,Counter-Strike,76561198068970227,28,1,10668,640,9906,1698261011,...,1,59,12,0.826206,0,0,0,0,1,NaN
4,147284743,10,Counter-Strike,76561199137893460,19,5,543,10,526,1697810991,...,1,128,24,0.853612,0,1,0,0,1,NaN


In [197]:
with open('combined_df.json', 'r') as f:
    sample_data = [json.loads(line) for line in f]

In [217]:
# Extract valid appids from the sample file
# valid_appids = {entry['appid'] for entry in sample_data}

# Keep original dataframe to preserve all game name mappings
df_all = df.copy()

# Filter df based on appids in the sample (for training)
# df = df[df['appid'].isin(valid_appids)]

# Build appid-to-game mapping from the full data to avoid KeyErrors later
df_games = df_all[['appid', 'game']].dropna().drop_duplicates(subset='appid')
appid2game = df_games.set_index('appid')['game'].to_dict()
game2appid = df_games.set_index('game')['appid'].to_dict()
# del df_games



# Keep only the relevant columns for ALS input
# df = df[['appid', 'author_steamid', 'voted_up']]


In [179]:
appid2genres = {entry['appid']: entry['genre'] for entry in sample_data}

In [218]:
appid2game # dictionary from appid to game name
# appid2genres # dictionary from appid to list of string genres
df.columns
# Index(['appid', 'author_steamid', 'voted_up'], dtype='object')

Index(['recommendationid', 'appid', 'game', 'author_steamid',
       'author_num_games_owned', 'author_num_reviews',
       'author_playtime_forever', 'author_playtime_last_two_weeks',
       'author_playtime_at_review', 'author_last_played', 'language', 'review',
       'timestamp_created', 'timestamp_updated', 'voted_up', 'votes_up',
       'votes_funny', 'weighted_vote_score', 'comment_count', 'steam_purchase',
       'received_for_free', 'written_during_early_access',
       'hidden_in_steam_china', 'steam_china_location'],
      dtype='object')

# Validation Methodology

In [188]:
# df_pos = df[df['voted_up'] == 1]
all_appids = set(df['appid'].unique())

df_pos = df[df['voted_up'] == 1].copy()

# 1) Build *global* mappings on the entire positive set
all_users = df_pos['author_steamid'].unique()
all_items = df_pos['appid'].unique()

user_ids     = {u: idx for idx, u in enumerate(all_users)}
item_ids     = {i: idx for idx, i in enumerate(all_items)}
user_ids_inv = {idx: u for u, idx in user_ids.items()}
item_ids_inv = {idx: i for i, idx in item_ids.items()}

# Create train/test split by user
def train_test_split_per_user(df, test_size=0.2, seed=42):
    np.random.seed(seed)
    train_rows, test_rows = [], []

    for user, user_df in tqdm(df.groupby('author_steamid'), desc="Splitting train/test by user"):
        if len(user_df) < 2:
            train_rows.append(user_df)
        else:
            user_train, user_test = train_test_split(user_df, test_size=test_size)
            train_rows.append(user_train)
            test_rows.append(user_test)

    train_df = pd.concat(train_rows)
    test_df = pd.concat(test_rows)
    return train_df, test_df

def sample_negatives(test_df, train_df, num_negatives=50):
    user_pos_train = train_df.groupby('author_steamid')['appid'].apply(set).to_dict()
    user_pos_test = test_df.groupby('author_steamid')['appid'].apply(set).to_dict()
    
    test_data = []

    for user in tqdm(user_pos_test, desc="Sampling negatives"):
        pos_items = user_pos_test[user]
        neg_items = list(all_appids - user_pos_train.get(user, set()) - pos_items)
        sampled_negatives = random.sample(neg_items, min(num_negatives, len(neg_items)))
        
        for pos_item in pos_items:
            test_data.append((user, pos_item, 1))
        for neg_item in sampled_negatives:
            test_data.append((user, neg_item, 0))
    
    return pd.DataFrame(test_data, columns=['author_steamid', 'appid', 'label'])

In [219]:
df_pos = df[df['voted_up'] == 1].copy()

# 1) Build *global* mappings on the entire positive set
all_users = df_pos['author_steamid'].unique()
all_items = df_pos['appid'].unique()

user_ids     = {u: idx for idx, u in enumerate(all_users)}
item_ids     = {i: idx for idx, i in enumerate(all_items)}
user_ids_inv = {idx: u for u, idx in user_ids.items()}
item_ids_inv = {idx: i for i, idx in item_ids.items()}

In [190]:
user_to_true_cache = {}

In [191]:
def dcg_at_k(relevance, k):
    relevance = np.array(relevance)[:k]
    if relevance.size == 0:
        return 0.0
    return np.sum(relevance / np.log2(np.arange(2, relevance.size + 2)))

def ndcg_at_k(preds, true_items, k):
    relevance = [1 if item in true_items else 0 for item in preds[:k]]
    ideal_relevance = sorted(relevance, reverse=True)
    dcg = dcg_at_k(relevance, k)
    idcg = dcg_at_k(ideal_relevance, k)
    return dcg / idcg if idcg > 0 else 0.0

def average_precision_at_k(preds, true_items, k):
    hits, score = 0, 0.0
    for i, item in enumerate(preds[:k]):
        if item in true_items:
            hits += 1
            score += hits / (i + 1)
    if not true_items:
        return 0.0
    return score / min(len(true_items), k)

def evaluate(test_data, recommender_fn, k=10, use_cache=False):
    global user_to_true_cache
    user_to_preds = defaultdict(list)

    if use_cache and 'user_to_true_cache' in globals():
        user_to_true = user_to_true_cache
    else:
        user_to_true = defaultdict(set)
        for _, row in tqdm(test_data.iterrows(), total=len(test_data), desc="Processing test data"):
            user, appid, label = row['author_steamid'], row['appid'], row['label']
            if label == 1:
                user_to_true[user].add(appid)
        if use_cache:
            user_to_true_cache = user_to_true

    n = 0
    for user in tqdm(user_to_true, desc="Generating predictions"):
        n += 1
        preds = recommender_fn(user)[:k]
        user_to_preds[user] = preds

    # print(f"Success rate: {successes / n:.2%}")
    # input()
    precisions, recalls, ndcgs, mapks = [], [], [], []

    for user in tqdm(user_to_true, desc="Calculating metrics"):
        true_items = user_to_true[user]
        pred_items = user_to_preds[user]

        num_hit = len(set(pred_items) & true_items)
        precision = num_hit / k
        recall = num_hit / len(true_items) if len(true_items) > 0 else 0
        ndcg = ndcg_at_k(pred_items, true_items, k)
        mapk = average_precision_at_k(pred_items, true_items, k)

        precisions.append(precision)
        recalls.append(recall)
        ndcgs.append(ndcg)
        mapks.append(mapk)

    results = {
        'Precision@K': np.mean(precisions),
        'Recall@K': np.mean(recalls),
        'NDCG@K': np.mean(ndcgs),
        'MAP@K': np.mean(mapks)
    }
    return results

# Model Experiments

## Popularity-Based

In [49]:
train_df, test_df = train_test_split_per_user(df_pos)
# test_data = sample_negatives(test_df, train_df)

Splitting train/test by user: 100%|██████████| 320412/320412 [00:10<00:00, 29292.56it/s]


In [51]:
# Count how many users liked each item in the training data
item_popularity = train_df['appid'].value_counts()

# Recommend the top-N popular items for all users
top_k = 10
top_items = item_popularity.head(top_k).index.tolist()

# For each user, generate recommendations
def recommend_popular(user_id):
    return top_items


In [57]:
train_df['label'] = train_df['voted_up'].astype(int)

In [ ]:
results = evaluate(test_df, recommend_popular, k=10)

for metric, value in results.items():
    print(f"{metric}: {value * 100:.2f}%")

Calculating metrics: 100%|██████████| 39976/39976 [00:00<00:00, 112625.84it/s]

Precision@K: 0.43%
Recall@K: 3.84%
NDCG@K: 1.99%
MAP@K: 1.17%


## Collaborative Filtering - Implicit ALS

In [192]:
from scipy.sparse import coo_matrix
import implicit

In [220]:
df

,recommendationid,appid,game,author_steamid,author_num_games_owned,author_num_reviews,author_playtime_forever,author_playtime_last_two_weeks,author_playtime_at_review,author_last_played,...,voted_up,votes_up,votes_funny,weighted_vote_score,comment_count,steam_purchase,received_for_free,written_during_early_access,hidden_in_steam_china,steam_china_location
0,147449116,10,Counter-Strike,76561199183984450,51,12,2548,0,2480,1696305457,...,1,99,2,0.889438,0,1,0,0,1,NaN
1,147374264,10,Counter-Strike,76561198099573060,226,13,2369,0,2361,1696096555,...,1,122,8,0.914834,0,1,0,0,1,NaN
2,147357703,10,Counter-Strike,76561199080026894,118,23,13501,212,12957,1697630734,...,1,599,20,0.968375,3,1,0,0,1,NaN
3,147345102,10,Counter-Strike,76561198068970227,28,1,10668,640,9906,1698261011,...,1,59,12,0.826206,0,0,0,0,1,NaN
4,147284743,10,Counter-Strike,76561199137893460,19,5,543,10,526,1697810991,...,1,128,24,0.853612,0,1,0,0,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
498089,93910882,997010,Police Simulator: Patrol Officers,76561198055740634,0,15,22402,420,103,1698784566,...,1,168,13,0.857843,8,1,0,1,0,NaN
498090,93909129,997010,Police Simulator: Patrol Officers,76561198107142741,0,69,18,0,18,1623946025,...,1,1417,1226,0.938828,30,1,0,1,0,NaN
498091,51109633,997060,Kotodama: The 7 Mysteries of Fujisawa,76561198091884692,1588,32,835,0,835,1559864576,...,1,66,14,0.847317,0,0,1,0,0,NaN
498092,48814832,997180,be you,76561198136664035,95,8,7,0,7,1549516192,...,1,57,7,0.838971,2,0,0,0,0,NaN


In [205]:
appid2game[item_ids_inv[3]]

'Cook, Serve, Delicious! 3?!'

In [208]:
appid2game[]

SyntaxError: invalid syntax (1845518453.py, line 1)

In [ ]:
item_ids_inv[45360]


KeyError: 45360

In [213]:
all_items[45360]

IndexError: index 45360 is out of bounds for axis 0 with size 27315

In [232]:
import implicit
import scipy.sparse as sparse

# Drop NaN columns
data = df_all.copy()
# data = df.dropna()
# data = data.copy()

all_users = data['author_steamid'].unique()
all_items = data['appid'].unique()

# Rebuild mappings based on filtered data
user_ids     = {u: idx for idx, u in enumerate(all_users)}
item_ids     = {i: idx for idx, i in enumerate(all_items)}
user_ids_inv = {idx: u for u, idx in user_ids.items()}
item_ids_inv = {idx: i for i, idx in item_ids.items()}


data['user_id'] = data['author_steamid'].map(user_ids).astype(int)
data['item_id'] = data['appid'].map(item_ids).astype(int)
data['rating'] = data['voted_up'].astype(int)
data = data[['user_id', 'item_id', 'rating']]
# The implicit library expects data as a item-user matrix so we
# create two matricies, one for fitting the model (item-user) 
# and one for recommendations (user-item)
sparse_item_user = sparse.csr_matrix((data['rating'], (data['item_id'], data['user_id'])))
sparse_user_item = sparse.csr_matrix((data['rating'], (data['user_id'], data['item_id'])))

# Initialize the als model and fit it using the sparse item-user matrix
model = implicit.als.AlternatingLeastSquares(factors=20, regularization=0.1, iterations=20)

# Calculate the confidence by multiplying it by our alpha value.
alpha_val = 15
data_conf = (sparse_item_user * alpha_val).astype('double')

#Fit the model
model.fit(data_conf)


#---------------------
# FIND SIMILAR ITEMS
#---------------------

# Find the 10 most similar to counter strike
# item_id = list(item_ids.values())[1]  # Get the first item_id
item_id = 2000
n_similar = 10
print(appid2game[item_ids_inv[item_id]])

# Use implicit to get similar items.
similar = model.similar_items(0, n_similar)

n = 0
# Print the names of our most similar games
for item in zip(*similar):
    idx, score = item
    print('idx', idx, score)
    if idx not in item_ids_inv:
        print('idx not in item_ids_inv', idx)
        continue
    game_id = item_ids_inv.get(idx, "Not in item_ids_inv")
    print('game_id', game_id)
    if game_id not in appid2game:
        print('game_id not in appid2game', game_id)
        continue
    game_name = appid2game.get(game_id, "Not in appid2game")
    print('game_name', game_name)
    print(game_name, score)
# print(n)
input()
#------------------------------
# CREATE USER RECOMMENDATIONS
#------------------------------

# Create recommendations for user with id 2025
user_id = list(user_ids.values())[0]
print(user_id)
# Use the implicit recommender.
user_items = sparse_user_item[user_id]  # Get only the row for the target user
recommended = model.recommend(user_id, user_items)
# recommended = model.recommend(user_id, sparse_user_item)

games = []
scores = []
# Get game names from ids
for item in zip(*recommended):
    idx, score = item
    # game_id = df.iloc[idx].item_id
    game_id = item_ids_inv.get(idx, "Unknown Game")
    game_name = appid2game.get(game_id, "Unknown Game")
    games.append(game_name)
    games.append(all_items[data.iloc[idx]['item_id']])
    games.append(data.game.loc[data.item_id == idx].iloc[0])
    scores.append(min(max(score, 0), 1))
    scores.append(score)

# Create a dataframe of game names and scores
recommendations = pd.DataFrame({'game': games, 'score': scores})

print(recommendations)

100%|██████████| 20/20 [00:03<00:00,  5.68it/s]


Demons Never Lie
idx 0 1.0
game_id 10
game_name Counter-Strike
Counter-Strike 1.0
idx 349 0.8808598
game_id 1024590
game_name Hentai Babes - Sport Lovers
Hentai Babes - Sport Lovers 0.8808598
idx 227159 0.8635216
idx not in item_ids_inv 227159
idx 352390 0.80494446
idx not in item_ids_inv 352390
idx 352393 0.8049444
idx not in item_ids_inv 352393
idx 352389 0.8049444
idx not in item_ids_inv 352389
idx 227116 0.80283034
idx not in item_ids_inv 227116
idx 149364 0.802055
idx not in item_ids_inv 149364
idx 131883 0.8019941
idx not in item_ids_inv 131883
idx 88832 0.80196214
idx not in item_ids_inv 88832
0


AttributeError: 'DataFrame' object has no attribute 'game'

In [ ]:
from scipy.sparse import coo_matrix
import implicit

In [226]:
appid2game.keys()

dict_keys([10, 100, 1000010, 1000030, 1000080, 1000280, 1000310, 1000360, 1000380, 1000410, 1000440, 1000700, 1000760, 1000830, 1001060, 1001140, 1001220, 1001370, 1001371, 1001490, 1001640, 1001860, 1001960, 1001970, 1002, 1002000, 1002100, 1002130, 1002200, 1002230, 1002270, 1002280, 1002290, 1002300, 1002310, 1002360, 1002430, 1002560, 1002850, 1002873, 1002930, 1002970, 1003090, 1003120, 1003160, 1003200, 1003310, 1003360, 1003400, 1003480, 1003490, 1003520, 1003590, 1003750, 1003800, 1003890, 100400, 100410, 1004150, 1004240, 1004270, 1004330, 1004490, 1004570, 1004750, 1004770, 1004860, 1004890, 1004891, 1004910, 1004917, 1004919, 1004920, 1005000, 1005050, 1005120, 1005130, 1005300, 1005340, 1005410, 1005420, 1005460, 1005480, 1005520, 1005550, 1005870, 1005910, 1005930, 1005950, 1005990, 1006140, 1006220, 1006270, 1006400, 1006460, 1006510, 1006800, 1006880, 1006930, 1007040, 1007350, 1007390, 1007400, 1007730, 1007800, 1007810, 1007840, 1007990, 1008010, 1008020, 1008080, 1008

In [ ]:
# Step 1: Build user/item mappings from the ENTIRE dataset (not just training)
all_user_ids = {uid: idx for idx, uid in enumerate(df_pos['author_steamid'].unique())}
all_item_ids = {iid: idx for idx, iid in enumerate(df_pos['appid'].unique())}
all_item_ids_inv = {v: k for k, v in all_item_ids.items()}

# Step 2: Add index columns to the entire dataset
df_pos['user_idx'] = df_pos['author_steamid'].map(all_user_ids)
df_pos['item_idx'] = df_pos['appid'].map(all_item_ids)

# Step 3: Now split AFTER indexing
train_df, test_df = train_test_split_per_user(df_pos)

# Step 4: Build matrix only from train data, but use global user/item shapes
item_user_matrix = coo_matrix(
    (np.ones(len(train_df)), (train_df['item_idx'], train_df['user_idx'])),
    shape=(len(all_item_ids), len(all_user_ids))
).tocsr()


Splitting train/test by user: 100%|██████████| 320412/320412 [00:13<00:00, 23090.54it/s]


In [ ]:
from scipy.sparse import coo_matrix
import implicit

/opt/homebrew/Caskroom/miniconda/base/envs/mgt_flask/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
df_pos['user_idx'] = df_pos['author_steamid'].map(user_ids)
df_pos['item_idx'] = df_pos['appid'].map(item_ids)


In [ ]:
# ,
model_als = implicit.als.AlternatingLeastSquares(factors=50, iterations=10)
model_als.fit(item_user_matrix)

# 6) Precompute transpose once
user_items_csr = item_user_matrix.T.tocsr()

100%|██████████| 10/10 [00:02<00:00,  4.02it/s]


In [ ]:
# 4) Ensure you have your popularity fallback defined
top_items = (
    train_df['appid']
    .value_counts()
    .head(10)
    .index
    .tolist()
)

def recommend_als(user_id, k=10):
    if user_id not in user_ids:
        return top_items  # fallback to popularity

    user_idx = user_ids[user_id]
    user_interactions = user_items_csr[user_idx]
    item_indices, _ = model_als.recommend(
        user_idx, user_interactions, N=k, filter_already_liked_items=True
    )
    return [item_ids_inv[i] for i in item_indices if i in item_ids_inv]

In [ ]:
test_df['label'] = test_df['voted_up'].astype(int)

In [ ]:
results = evaluate(test_df, recommend_als, k=10, use_cache=False)

for metric, value in results.items():
    print(f"{metric}: {value:.4f}")

Generating predictions:   0%|          | 0/39976 [00:58<?, ?it/s]


IndexError: index 54547 is out of bounds for axis 0 with size 22903

In [171]:
len(item_ids)

27315

In [134]:
print(max(item_ids.values()))
print(max(data['item_id']))

27314
27314


In [172]:
len(appid2game)

27315

In [116]:
# Find rows with non-finite values
non_finite_values = data[~np.isfinite(data).all(axis=1)]

# Display rows with non-finite values
print(non_finite_values)

Empty DataFrame
Columns: [appid, author_steamid, voted_up]
Index: []


In [98]:
# Step 1: Build user/item mappings from the FULL dataset
all_user_ids = {uid: idx for idx, uid in enumerate(df_pos['author_steamid'].unique())}
all_item_ids = {iid: idx for idx, iid in enumerate(df_pos['appid'].unique())}
all_item_ids_inv = {v: k for k, v in all_item_ids.items()}
user_ids = all_user_ids
item_ids = all_item_ids
item_ids_inv = all_item_ids_inv

# Step 2: Add index columns to full dataset
df_pos['user_idx'] = df_pos['author_steamid'].map(user_ids)
df_pos['item_idx'] = df_pos['appid'].map(item_ids)

# Step 3: Split for evaluation purposes only (recommendations now trained on full set)
train_df, test_df = train_test_split_per_user(df_pos)

Splitting train/test by user: 100%|██████████| 320412/320412 [00:21<00:00, 14805.55it/s]


In [101]:
df

,appid,author_steamid,voted_up
0,10,76561199183984450,1
1,10,76561198099573060,1
2,10,76561199080026894,1
3,10,76561198068970227,1
4,10,76561199137893460,1
...,...,...,...
498089,997010,76561198055740634,1
498090,997010,76561198107142741,1
498091,997060,76561198091884692,1
498092,997180,76561198136664035,1


In [100]:
user_items_csr.shape

(320412, 22903)

In [88]:
max(user_ids.values())

320411

In [ ]:
# Step 4: Build item-user matrix from ALL data
item_user_matrix = coo_matrix(
    (np.ones(len(df_pos)), (df_pos['item_idx'], df_pos['user_idx'])),
    shape=(len(item_ids), len(user_ids))
).tocsr()

# Step 5: Train ALS model
model_als = implicit.als.AlternatingLeastSquares(factors=50, regularization=0.1, iterations=10)
alpha_val = 15
confidence_matrix = (item_user_matrix * alpha_val).astype('double')
model_als.fit(confidence_matrix)

# Step 6: Precompute transpose for recommendations
user_items_csr = item_user_matrix.T.tocsr()

# Step 7: Define fallback for unseen users (should rarely be used now)
top_items = (
    df_pos['appid']
    .value_counts()
    .head(10)
    .index
    .tolist()
)
# Step 8: Define recommendation function (no more bounds errors!)
def recommend_als(user_id, k=10):
    if user_id not in user_ids:
        return top_items

    user_idx = user_ids[user_id]
    user_interactions = user_items_csr[user_idx]
    user_interactions = user_interactions.toarray().reshape(-1)
    print('len user interactions', len(user_interactions))
    print('user idx', user_idx)
    
    item_indices, _ = model_als.recommend(
        user_idx, user_interactions, N=k, filter_already_liked_items=False
    )
    return [item_ids_inv[i] for i in item_indices if i in item_ids_inv]


100%|██████████| 10/10 [00:02<00:00,  3.74it/s]


In [91]:

user_vecs = model_als.user_factors
item_vecs = model_als.item_factors


In [86]:
# Step 9: Evaluate
test_df['label'] = test_df['voted_up'].astype(int)
results = evaluate(test_df, recommend_als, k=10, use_cache=False)

for metric, value in results.items():
    print(f"{metric}: {value:.4f}")

Generating predictions:   0%|          | 0/39976 [00:00<?, ?it/s]

len user interactions 22903
user idx 54547


IndexError: index 54547 is out of bounds for axis 0 with size 22903

In [74]:

# Step 4: Create item-user matrix (REQUIRED format for implicit)
item_user_matrix = coo_matrix(
    (np.ones(len(train_df)), (train_df['item_idx'], train_df['user_idx'])),
    shape=(len(item_ids), len(user_ids))
).tocsr()

# Step 5: Initialize and train ALS model with confidence
model_als = implicit.als.AlternatingLeastSquares(factors=50, regularization=0.1, iterations=10)
alpha_val = 15
confidence_matrix = (item_user_matrix * alpha_val).astype('double')
model_als.fit(confidence_matrix)

# Step 6: Transpose for recommendation usage
user_items_csr = item_user_matrix.T.tocsr()

# Step 7: Define fallback for cold-start users
top_items = (
    train_df['appid']
    .value_counts()
    .head(10)
    .index
    .tolist()
)

# Step 8: Wrap recommendation logic with fixed compatibility
def recommend_als(user_id, k=10):
    if user_id not in user_ids:
        return top_items  # fallback to popularity

    user_idx = user_ids[user_id]
    user_interactions = user_items_csr[user_idx]
    
    item_indices, _ = model_als.recommend(
        user_idx, user_interactions, N=k, filter_already_liked_items=True
    )
    
    return [item_ids_inv[i] for i in item_indices if i in item_ids_inv]

100%|██████████| 10/10 [00:02<00:00,  4.06it/s]


In [75]:
# Step 9: Evaluate results
test_df['label'] = test_df['voted_up'].astype(int)
results = evaluate(test_df, recommend_als, k=10, use_cache=False)

for metric, value in results.items():
    print(f"{metric}: {value:.4f}")

Generating predictions:   0%|          | 0/39976 [00:00<?, ?it/s]


IndexError: index 54547 is out of bounds for axis 0 with size 22903

In [64]:
# Step 1: Build user/item mappings from the ENTIRE dataset (not just training)
all_user_ids = {uid: idx for idx, uid in enumerate(df_pos['author_steamid'].unique())}
all_item_ids = {iid: idx for idx, iid in enumerate(df_pos['appid'].unique())}
all_item_ids_inv = {v: k for k, v in all_item_ids.items()}

# Step 2: Add index columns to the entire dataset
df_pos['user_idx'] = df_pos['author_steamid'].map(all_user_ids)
df_pos['item_idx'] = df_pos['appid'].map(all_item_ids)

# Step 3: Now split AFTER indexing
train_df, test_df = train_test_split_per_user(df_pos)

# Step 4: Build matrix only from train data, but use global user/item shapes
item_user_matrix = coo_matrix(
    (np.ones(len(train_df)), (train_df['item_idx'], train_df['user_idx'])),
    shape=(len(all_item_ids), len(all_user_ids))
).tocsr()


Splitting train/test by user: 100%|██████████| 320412/320412 [00:13<00:00, 23090.54it/s]


In [ ]:
from scipy.sparse import coo_matrix
import implicit

/opt/homebrew/Caskroom/miniconda/base/envs/mgt_flask/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
df_pos['user_idx'] = df_pos['author_steamid'].map(user_ids)
df_pos['item_idx'] = df_pos['appid'].map(item_ids)


In [65]:
# ,
model_als = implicit.als.AlternatingLeastSquares(factors=50, iterations=10)
model_als.fit(item_user_matrix)

# 6) Precompute transpose once
user_items_csr = item_user_matrix.T.tocsr()

100%|██████████| 10/10 [00:02<00:00,  4.02it/s]


In [ ]:
# 4) Ensure you have your popularity fallback defined
top_items = (
    train_df['appid']
    .value_counts()
    .head(10)
    .index
    .tolist()
)

def recommend_als(user_id, k=10):
    if user_id not in user_ids:
        return top_items  # fallback to popularity

    user_idx = user_ids[user_id]
    user_interactions = user_items_csr[user_idx]
    item_indices, _ = model_als.recommend(
        user_idx, user_interactions, N=k, filter_already_liked_items=True
    )
    return [item_ids_inv[i] for i in item_indices if i in item_ids_inv]

In [68]:
test_df['label'] = test_df['voted_up'].astype(int)

In [72]:
results = evaluate(test_df, recommend_als, k=10, use_cache=False)

for metric, value in results.items():
    print(f"{metric}: {value:.4f}")

Generating predictions:   0%|          | 0/39976 [00:58<?, ?it/s]


IndexError: index 54547 is out of bounds for axis 0 with size 22903

## Content-Aware: LightFM With Genres

In [ ]:
train_df, test_df = train_test_split_per_user(df_pos)  # same as before

Splitting train/test by user: 100%|██████████| 320412/320412 [00:11<00:00, 28447.29it/s]


In [19]:
from lightfm import LightFM
from lightfm.data import Dataset

dataset = Dataset()

# Fit users, items, and item features (genres)
dataset.fit(train_df['author_steamid'], appid2genres.keys())
all_genres = set(g for genres in appid2genres.values() for g in genres)
dataset.fit_partial(items=train_df['appid'], item_features=all_genres)

# Build interactions
interactions, _ = dataset.build_interactions(
    ((row['author_steamid'], row['appid']) for _, row in train_df.iterrows())
)

# Build item features
item_features = dataset.build_item_features(
    ((appid, appid2genres.get(appid, [])) for appid in appid2genres)
)

In [20]:
# Train model
model_lfm = LightFM(loss='warp')
model_lfm.fit(interactions, item_features=item_features, epochs=10)


In [21]:
def recommend_lightfm(user_id, k=10):
    if user_id not in user_ids:
        return top_items  # fallback

    user_idx = user_ids[user_id]
    scores = model_lfm.predict(user_ids[user_id], np.arange(len(item_ids)), item_features=item_features)
    top_indices = np.argsort(-scores)[:k]
    return [item_ids_inv[i] for i in top_indices]


In [23]:
results = evaluate(test_data, recommend_lightfm, k=10)

for metric, value in results.items():
    print(f"{metric}: {value:.4f}")

Generating predictions: 100%|██████████| 39976/39976 [01:15<00:00, 531.70it/s]


Success rate: 100.00%


Calculating metrics: 100%|██████████| 39976/39976 [00:00<00:00, 110374.24it/s]

Precision@K: 0.0001
Recall@K: 0.0004
NDCG@K: 0.0003
MAP@K: 0.0002
